In [1]:
import pandas as pd
import numpy as np

In [2]:
# 데이터 로드
all_train = pd.read_parquet("./data/train.parquet", engine="pyarrow")
test = pd.read_parquet("./data/test.parquet", engine="pyarrow").drop(columns=['ID'])

print("Train shape:", all_train.shape)
print("Test shape:", test.shape)

Train shape: (10704179, 119)
Test shape: (1527298, 118)


In [3]:
# 기존 테이블
all_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10704179 entries, 0 to 10704178
Columns: 119 entries, gender to clicked
dtypes: float32(112), int32(1), object(6)
memory usage: 5.0+ GB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527298 entries, 0 to 1527297
Columns: 118 entries, gender to history_b_30
dtypes: float32(112), object(6)
memory usage: 722.4+ MB


### 메모리 사용량 최적화 (Data Type Downcasting)

In [5]:
def reduce_mem_usage(df, verbose=True):
    """
    데이터프레임의 메모리 사용량을 줄이는 함수.
    
    Args:
        df (pd.DataFrame): 최적화할 데이터프레임
        verbose (bool): 최적화 과정 및 결과를 출력할지 여부
        
    Returns:
        pd.DataFrame: 메모리 사용량이 최적화된 데이터프레임
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                # 정수형 타입 다운캐스팅
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # 실수형 타입 다운캐스팅
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
                    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Mem. usage decreased to {end_mem:5.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    return df

# --- 데이터 로드 및 최적화 실행 ---
print("Loading train data...")
train_df = reduce_mem_usage(all_train)

print("\nLoading test data...")
test_df = reduce_mem_usage(test)

Loading train data...
Mem. usage decreased to 2827.70 Mb (44.6% reduction)

Loading test data...
Mem. usage decreased to 402.01 Mb (44.4% reduction)


In [6]:
# 최적화된 데이터프레임을 새로운 Parquet 파일로 저장
train_df.to_parquet('./data/train_optimized.parquet', index=False)
test_df.to_parquet('./data/test_optimized.parquet', index=False)